In [5]:
import pandas as pd
import requests

def get_bok_df(info, classification,start_data, end_data, specific_info):
    apikey = "HF0ML1Y9A0RN7R7K4JE2"
    url = 'https://ecos.bok.or.kr/api/StatisticSearch/' + apikey + '/json/kr/1/100/817Y002/D/20180101/20221231/010200000'
    response = requests.get(url)

    result = response.json()
    list_total_count=(int)(result['StatisticSearch']['list_total_count'])
    list_count=(int)(list_total_count/100) + 1
    rows=[]
    for i in range(0,list_count):
        start = str(i * 100 + 1)
        end = str((i + 1) * 100)
        url = 'https://ecos.bok.or.kr/api/StatisticSearch/' + apikey + '/json/kr/' \
                + start + '/' + end + f"/{info}/{classification}/{start_data}/{end_data}/{specific_info}" 
        response = requests.get(url)
        result = response.json()
        rows = rows + result['StatisticSearch']['row']
    df=pd.DataFrame(rows)
    return df


정보 가져오기

In [71]:
# 국고채 3년물 금리 일단위 -> 집계필요
param_kor_govrn_3ybond = ['817Y002', 'D', 20180101, 20221231, '010200000'] 

# 원화의 대미 달러 종가기준 연평균
param_uskor_exchange = ['731Y006', 'A', 2018, 2022, '0000003']

# M2 경제주체별 연평균, 단위 십억원
param_m2 = ['101Y015', 'A', 2018, 2022, 'BBHA00']

# 생산자물가지수 연평균, 2015년 PPI를 100으로 함.
param_PPI = ['404Y014', 'A', 2018, 2022, '*AA']

# 한국 정책금리 월단위
param_kr_policyratio = ['902Y006', 'M', 201801, 202212, 'KR']
# 미국 정책금리 월단위
param_us_policyratio = ['902Y006', 'M', 201801, 202212, 'US']
# # 한국은행 기준금리 일단위
# param_kor_standard_yield = ['722Y001', 'D', 20180101, 20221231, '0101000']
